# Rapport : Classification de Commentaires

## Table des Matières
1. [Contexte et Presentation du Projet](#1-contexte-et-presentation-du-projet)
2. [Jeu de Donnees et Pretraitement](#2-jeu-de-donnees-et-pretraitement)
   - [Lecture du Jeu de Donnees](#21-lecture-du-jeu-de-donnees)
   - [Creation d'un Label Binaire](#22-creation-dun-label-binaire)
   - [Nettoyage des commentaires](#23-nettoyage-des-commentaires)
   - [Suppression des Stopwords](#24-suppression-des-stopwords)
   - [Tokenization et Padding des textes](#25-tokenization-et-padding-des-textes)
3. [Approche Utilisee](#3-approche-utilisee)
   - [Modele Simple](#31-modele-simple)
   - [Modele Final avec LSTM](#32-modele-final-avec-lstm)
4. [Entrainement du Modele](#4-entrainement-du-modele)
5. [Evaluation du Modele](#5-evaluation-du-modele)
6. [Ameliorations Possibles](#6-ameliorations-possibles)
7. [Pipeline de Classification](#7-pipeline-de-classification)
8. [Conclusion](#8-conclusion)



## 1. Contexte et Presentation du Projet

Les entreprises proposant des plateformes interactives où les utilisateurs peuvent laisser des commentaires sont confrontées à un défi majeur : modérer les contenus publiés. En effet, elles peuvent être tenues responsables des propos toxiques ou injurieux laissés par leurs utilisateurs. L'objectif de ce projet est donc de développer un modèle capable d’analyser automatiquement des commentaires et de les classifier en différentes catégories afin de faciliter la modération et d’améliorer l’expérience utilisateur.

La classification des commentaires est un problème du traitement automatique du langage naturel (**NLP**). Ce domaine de l’intelligence artificielle permet d’extraire du sens à partir de textes et de prendre des décisions automatisées en fonction du contexte. Dans ce projet, l'objectif est d’identifier les commentaires toxiques en se basant sur des caractéristiques linguistiques et syntaxiques.

## 2. Jeu de Donnees et Pretraitement

Le projet utilise le jeu de données du **Jigsaw Toxic Comment Classification Challenge**, qui contient des commentaires annotés selon leur nature toxique. Les classes possibles comprennent des catégories telles que :

- **Toxic** : Commentaire général toxique
- **Severe toxic** : Commentaire particulièrement agressif
- **Obscene** : Contenu obscène
- **Threat** : Contenu menaçant
- **Insult** : Insulte directe
- **Identity hate** : Discours haineux basé sur l’identité

### **Importation**


In [ ]:
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


### **Prétraitement des données**

Avant d'entraîner un modèle de classification, il est essentiel de nettoyer et de transformer les données afin d’optimiser leur exploitation. 
Mais nous allons d'abords créer un label binaire indiquant si un commentaire est toxique (si l'une des colonnes de toxicité vaut 1).

### 2.1 Lecture du Jeu de Donnees

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')
print(data.head())

### 2.2 Creation d'un Label Binaire

On considère qu'un commentaire est toxique si au moins l'une des colonnes de toxicité vaut 1.

In [ ]:
# Liste des colonnes représentant les différentes catégories de toxicité
toxic_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Création d'une nouvelle colonne 'label' dans le DataFrame, qui prend la valeur maximale parmi les colonnes de toxicité pour chaque ligne
# Cela permet de marquer un commentaire comme toxique si l'une des catégories de toxicité a la valeur la plus élevée (1)
data['label'] = data[toxic_columns].max(axis=1)

# Affichage des premières lignes du DataFrame avec les colonnes 'comment_text' et 'label' pour vérifier que les résultats sont correctement associés aux commentaires
print(data[['comment_text', 'label']].head())

### 2.3 Nettoyage des commentaires

Suppression des caractères spéciaux, conversion en minuscules

In [ ]:
# Définition de la fonction de nettoyage du texte
def clean_text(text):
    # Conversion du texte en minuscules et suppression des ponctuations
    text = text.lower()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    return text

# Application de la fonction 'clean_text'
data['comment_clean'] = data['comment_text'].apply(clean_text)

# Affichage
print(data['comment_clean'].head())

exemple de nettoyage des commentaires  estce que ça fonctionne 


### 2.4 Suppression des Stopwords

Utilisation de NLTK pour retirer les mots vides (stopwords).

In [ ]:
nltk.download('punkt')      # Tokenisation des mots
nltk.download('stopwords')  # Liste des mots vides

# Fonction pour supprimer les mots vides d'un texte
def remove_stopwords(text):
    tokens = word_tokenize(text)  # Découpe du texte en mots
    stop_words = set(stopwords.words('french'))  # Liste des mots vides
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Filtrage des mots vides
    return ' '.join(filtered_tokens)  # Réassemble les mots restants

# Application de la fonction
data['comment_clean'] = data['comment_clean'].apply(remove_stopwords)

# Affichage des premières lignes
print(data['comment_clean'].head())

### 2.5 Tokenization et Padding des textes

Transformation des phrases en une liste de mots exploitables par le modèle puis applique un padding pour uniformiser leur longueur.


In [ ]:
# Définition du nombre maximum de mots et de la longueur maximale des séquences
max_words = 10000
max_len = 100

# Initialisation du Tokenizer avec un nombre maximum de mots et un token pour les mots inconnus
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
# Apprentissage du Tokenizer sur le texte nettoyé
tokenizer.fit_on_texts(data['comment_clean'])
# Conversion des textes
sequences = tokenizer.texts_to_sequences(data['comment_clean'])
# Application du padding pour uniformiser la longueur des séquences
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# Affichage
print("Shape des données après padding :", padded_sequences.shape)

## 3. Approche Utilisee

### 3.1 Modele Simple

Dans un premier temps, un modèle de base a été entraîné pour évaluer la difficulté du problème. Celui-ci utilisait un simple réseau de neurones denses avec une couche d'**embedding**. Ce modèle initial permet de comprendre si les données sont linéairement séparables et d'établir une ligne de base pour la performance attendue.



In [ ]:
# Définition du modèle simple de réseau de neurones avec Keras
model_simple = Sequential([
    # Couche d'embedding pour transformer les indices des mots en vecteurs
    Embedding(input_dim=10000, output_dim=16, input_length=20),
    # Aplatissement des vecteurs en une seule dimension (Vecteur 1D)
    Flatten(),
    # Couche dense avec 16 neurones et activation ReLU
    Dense(16, activation='relu'),
    # Couche de sortie avec un neurone et activation sigmoïde pour une classification binaire
    Dense(1, activation='sigmoid')
])
model_simple.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_simple.summary()

### 3.2 Modele Final

Progressivement, nous avons amélioré le modèle en intégrant des techniques avancées du deep learning, notamment :

- Une couche **Embedding** permettant de mieux représenter les mots en fonction de leur contexte.
- Deux couches **LSTM (Long Short-Term Memory)** capables de capturer les relations entre les mots et de gérer la dépendance à long terme entre eux.
- Des couches **Dropout** pour éviter l’overfitting et assurer une meilleure généralisation du modèle.
- Une couche **Dense** avec activation **sigmoïde** pour produire une probabilité indiquant si un commentaire est toxique ou non.

In [ ]:
# Définition du modèle de réseau de neurones avec LSTM pour la classification binaire
model_final = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=20),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model_final.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_final.summary()

L'utilisation des LSTM est particulièrement adaptée aux tâches de classification de texte, car elles permettent de mieux prendre en compte la structure et la signification des phrases.

## 4. Entrainement du modele

Le modèle final a été entraîné sur **80%** des données et testé sur **20%**. 

In [ ]:
# Entraînement du modèle avec les données
X = padded_sequences
y = np.array(data['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
history = model_final.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

## 5. Evaluation du modele

Plusieurs métriques d’évaluation ont été utilisées :

- **Accuracy** : Mesure la proportion de classifications correctes.
- **F1-score** : Prend en compte la précision et le rappel, essentiel dans un contexte de données déséquilibrées.
- **Matrice de confusion** : Analyse les faux positifs et faux négatifs pour mieux comprendre les erreurs du modèle.
Exemple de code pour évaluer le modèle avec scikit-learn :

In [ ]:
# Boucle pour évaluer le modèle pour chaque type de toxicité
for toxic_type in toxic_columns:
    print(f"Évaluation pour la catégorie : {toxic_type}")
    loss, accuracy = model_final.evaluate(X_test, y_test, verbose=0)
    print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    # Prédiction des résultats sur l'ensemble de test (binaire : 0 ou 1)
    y_pred = (model_final.predict(X_test) > 0.5).astype("int32")
    # Affichage du rapport de classification pour chaque catégorie
    print(classification_report(y_test, y_pred))
    # Affichage de la matrice de confusion pour évaluer les prédictions
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    # Séparation des évaluations de chaque catégorie
    print("\n" + "-"*50 + "\n")

Les résultats obtenus montrent une bonne capacité du modèle à détecter les commentaires toxiques tout en limitant les faux positifs. Toutefois, certaines erreurs persistent, notamment dans la différenciation entre certaines catégories proches, comme *insult* et *obscene*.

## 6. Ameliorations Possibles

Plusieurs améliorations pourraient être apportées pour affiner la classification et améliorer la précision du modèle :

- **Utilisation de modèles pré-entraînés** : L’intégration de modèles avancés comme **BERT** ou **GPT** pourrait permettre une meilleure compréhension du langage et améliorer la classification.
- **Augmentation des données** : Générer des variations de commentaires existants pour enrichir l’apprentissage du modèle.
- **Optimisation des hyperparamètres** : Ajuster les paramètres tels que la taille des embeddings, le nombre de neurones par couche, et le taux de dropout pour obtenir de meilleures performances.
- **Filtrage plus avancé des données** : Supprimer les commentaires ambigus ou mal étiquetés pour améliorer la qualité des données d’entraînement.
- **Entraînement multi-catégories** : Modifier l’architecture du modèle pour classer les commentaires dans plusieurs catégories en même temps.

## 7. Pipeline de Classification

Nous avons mis en place une pipeline permettant de classifier une phrase brute saisie par l'utilisateur.  
Cette pipeline suit ces étapes :
1. Nettoyage du texte (mise en minuscules, suppression de la ponctuation)
2. Suppression des stopwords
3. Tokenization et conversion en séquence numérique
4. Application d'un padding
5. Passage dans le modèle entraîné pour obtenir une prédiction

### **Implementation de la Pipeline**

In [ ]:
# Fonction de prétraitement du texte et de prédiction de la toxicité
def preprocess_and_predict(text, tokenizer, model, max_len=100):
    # Nettoyage du texte
    text_cleaned = clean_text(text)
    # Stopwords
    text_cleaned = remove_stopwords(text_cleaned)
    # Conversion du texte en séquence d'indices de mots
    sequence = tokenizer.texts_to_sequences([text_cleaned])
    # Padding
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')
    # Prédiction
    prediction = model.predict(padded_sequence)
    # Retourne le résultat : "Toxique" si la probabilité est supérieure à 0.5, sinon "Non toxique"
    return "Toxique" if prediction > 0.5 else "Non toxique"

# Exemple de prédiction sur un commentaire
exemple = "Je te déteste, tu es horrible !"
# Prédiction du modèle
resultat = preprocess_and_predict(exemple, tokenizer, model_final)  
print(f"Le commentaire est : {resultat}")


## 8. Conclusion

Ce projet m'a permis de développer une solution efficace pour détecter automatiquement les commentaires toxiques sur une plateforme. L’utilisation des **LSTM** et du **deep learning** a montré des résultats prometteurs, bien qu’il reste encore des marges d’amélioration. 

L’approche développée ici pourrait être étendue à d’autres domaines nécessitant une modération automatisée du contenu, tels que :

- Les réseaux sociaux
- Les forums en ligne
- Les plateformes de e-commerce

L’implémentation de solutions plus avancées, intégrant des modèles de NLP pré-entraînés, pourrait permettre d’optimiser davantage la détection des propos toxiques et d’améliorer la sûreté des plateformes en ligne.